In [1]:
# https://ppiazi.tistory.com/entry/python-cyclomatic-complexity-%EB%AA%A8%EB%93%88-lizard

In [2]:
# https://github.com/terryyin/lizard

In [3]:
# pip install cognitive_complexity

In [4]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
sys.path.insert(1, p)
sys.path.insert(1, pp)

In [5]:
#customized preprocessing pkg
import preprocess as pp
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
# https://joaorafaelm.github.io/notebook/python/2021/04/10/cognitive-complexity.html
import config.config as conf
from lib.complexity import SrcBulkInsert as SB

In [6]:
input_year = '2023'

In [7]:
input_pl = 'java'

In [8]:
print(conf.DATA_PATH)

/usr/share/d_ollama


### 소스코드 complexity 계산

In [9]:
# 파일 불러오기
with open(f'{conf.DATA_PATH}/data/complexity_{input_pl}_df_{input_year}.pkl', 'rb') as f:
    complexity_java_df = pickle.load(f)

In [10]:
complexity_java_df['cdate'].min()

'2023-01-01'

In [11]:
complexity_java_df['cdate'].max()

'2023-12-03'

In [12]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()

In [13]:
complexity_java_df['c_h_body'] = complexity_java_df['body'].apply(lambda x : htmlp.clean_html_str(x))

In [14]:
complexity_java_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48292 entries, 0 to 48291
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        48292 non-null  int64 
 1   cdate     48292 non-null  object
 2   title     48292 non-null  object
 3   body      48292 non-null  object
 4   c_h_body  48292 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.8+ MB


In [15]:
complexity_java_df['c_body'] = complexity_java_df['body'].apply(lambda x : codep(x))

In [16]:
complexity_java_df

,id,cdate,title,body,c_h_body,c_body
0,77492206,2023-11-16,Gson doesn't properly convert a LinkedHashMap ...,<p>When I convert a LinkedHashMap directly to ...,When I convert a LinkedHashMap directly to Jso...,"{'code_sections': [{'off_beg': 274, 'off_end':..."
1,77593745,2023-12-03,Native library loading issue in java 11,<p>I have migrated my java application from 8 ...,I have migrated my java application from 8 to ...,{'code_sections': []}
2,77593782,2023-12-03,Converting from JDBC to JPA: Unit tests failin...,<p>How does one get unit tests to use entities...,How does one get unit tests to use entities af...,"{'code_sections': [{'off_beg': 753, 'off_end':..."
3,77593813,2023-12-03,Multiple Hexagonal architecture with sprint boot,<p>I Work on spring boot application with spri...,I Work on spring boot application with spring ...,{'code_sections': []}
4,77591164,2023-12-02,Package a JavaFX Application as a Native Execu...,"<p>Based on <a href=""https://inside.java/2023/...","Based on this article, and with a default Java...",{'code_sections': []}
...,...,...,...,...,...,...
48287,77356120,2023-10-25,entityManagerFactory bean creation issue,<p>I am working migrating an application from ...,I am working migrating an application from Jav...,{'code_sections': []}
48288,77356158,2023-10-25,"JLabel Not Showing up when i run the program, ...",<p>I am using Java GUI builder in NetBeans(dra...,I am using Java GUI builder in NetBeans(drag a...,"{'code_sections': [{'off_beg': 205, 'off_end':..."
48289,77356182,2023-10-25,In Spring Data JPA how do I perform a `SELECT ...,<p>So in Spring Data JPA I can do the followin...,So in Spring Data JPA I can do the following\n...,"{'code_sections': [{'off_beg': 63, 'off_end': ..."
48290,77356226,2023-10-25,How to throw exception in main thread when Com...,"<p>When my service is called, I have 2 <code>@...","When my service is called, I have 2 @Async Fei...",{'code_sections': []}


In [17]:
complexity_java_src = complexity_java_df[['id', 'c_body']]

In [18]:
complexity_java_src['code_len'] = complexity_java_src['c_body'].apply(lambda x: len(x['code_sections']))

/tmp/ipykernel_2617434/804880781.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complexity_java_src['code_len'] = complexity_java_src['c_body'].apply(lambda x: len(x['code_sections']))


In [19]:
complexity_java_src = complexity_java_src[complexity_java_src['code_len']>0]

In [20]:
complexity_java_src.head(5)

,id,c_body,code_len
0,77492206,"{'code_sections': [{'off_beg': 274, 'off_end':...",2
2,77593782,"{'code_sections': [{'off_beg': 753, 'off_end':...",2
5,77591191,"{'code_sections': [{'off_beg': 11, 'off_end': ...",1
6,77591209,"{'code_sections': [{'off_beg': 237, 'off_end':...",1
7,77591269,"{'code_sections': [{'off_beg': 319, 'off_end':...",2


In [21]:
src_df = pd.DataFrame(columns = ['id', 'src'])
def code_explode(row) : 
    global src_df
    src_arr = []
    for i in range(row['code_len']):
        new_rows = pd.DataFrame({'id': [row['id']], 'src': [row['c_body']['code_sections'][i]['span_str']]})
        src_df = pd.concat([src_df, new_rows], axis=0)

    return True
        

In [22]:
# bert_src_ex = bert_src[['q_id', 'q_prep_text']].apply(pd.Series.explode)
complexity_java_src.apply(code_explode, axis = 1)

0        True
2        True
5        True
6        True
7        True
         ... 
48283    True
48284    True
48285    True
48288    True
48289    True
Length: 33285, dtype: bool

In [23]:
src_df.head()

,id,src
0,77492206,public class Example {\n public static void...
0,77492206,"GSON HashMap!\n{\n ""60"": ""data1"",\n ""25"": ""d..."
0,77593782,AccountJsonTest >; accountSerializationTest() ...
0,77593782,plugins {\n id 'java'\n id 'org.springfr...
0,77591191,public class test2 {\n public static void ...


In [24]:
src_df.reset_index(drop=True, inplace=True)

In [25]:
src_df.reset_index(inplace=True)

In [26]:
src_df.head()

,index,id,src
0,0,77492206,public class Example {\n public static void...
1,1,77492206,"GSON HashMap!\n{\n ""60"": ""data1"",\n ""25"": ""d..."
2,2,77593782,AccountJsonTest >; accountSerializationTest() ...
3,3,77593782,plugins {\n id 'java'\n id 'org.springfr...
4,4,77591191,public class test2 {\n public static void ...


In [27]:
sbi = SB.SrcBulkInsert()
b_re = sbi.get_from_db(src_df.iloc[0, 0], src_df.iloc[0, 1])

if b_re==False :
   sbi.insert_posts_src(src_df)

In [ ]:
for i in range(src_df.shape[0]):
    idx = src_df.iloc[i, 0]
    qid = src_df.iloc[i, 1]
    src = src_df.iloc[i, 2]
    file_nm = str(idx)+'_' + str(qid)+'.java'
    with open(f'{conf.DATA_PATH}/data/src/{input_year}_{input_pl}_src/{file_nm}', 'wb') as f:
        f.write(src.encode())


In [ ]:
# https://github.com/rohaquinlop/complexipy

In [ ]:
# pip install complexipy

In [ ]:
# !complexipy /home/mghan/sopjt/git/data/src/3_74963438.py -l file -o

In [ ]:
# cyclomatic은 모든 언어 가능 but cognitive는 언어별로 따로 분석해야함
# 하려면 소나큐브 사야 함 
# pyhon 기준으로는 조금 삽질하면 할 수 있을 듯.... 해보고 유의미하면 소나큐브 고?


### complexity 결과 확인

In [ ]:
path = f'{conf.DATA_PATH}/data/src/{input_year}_{input_pl}_excel/'
file_list = os.listdir(path)
file_list_csv = [file for file in file_list if file.endswith(".csv")]

# print ("file_list_csv: {}".format(file_list_csv))

In [ ]:
calc_com_df = pd.DataFrame(columns = ['Absolute Module Path','Module Position',	'Module Declaration',	'Max nesting',	'Cognitive Complexity'])
for file_nm in file_list_csv :
    tmp = pd.read_csv(path+file_nm, sep=';')
    tmp['file_nm'] = file_nm
    calc_com_df = pd.concat([calc_com_df, tmp], axis = 0)

In [ ]:
calc_com_df.head()

In [ ]:
# 전체 데이터셋 
with open(f'{conf.DATA_PATH}/data/calc_com_df_{input_year}_{input_pl}.pkl', 'wb') as f:
    pickle.dump(calc_com_df, f)

In [ ]:
calc_com_df.head()

In [ ]:
calc_com_df.reset_index(drop=True, inplace=True)

In [ ]:
calc_com_df.head()

In [ ]:
calc_com_df['index'] = calc_com_df['file_nm'].apply(lambda x : x.split('_')[0])
# calc_com_df['Path'].apply(lambda x : [  for i in x.split('_')])

In [ ]:
calc_com_df['id'] = calc_com_df['file_nm'].apply(lambda x : x.split('_')[1].split('.')[0])

In [ ]:
calc_com_df.sort_values(by = ['index', 'id'])

In [ ]:
calc_com_df[['index', 'id', 'Cognitive Complexity']] = calc_com_df[['index', 'id', 'Cognitive Complexity']].astype('int')

In [ ]:
src_df[['index', 'id']] = src_df[['index', 'id']].astype('int')

In [ ]:
calc_com_df.info()

In [ ]:
calc_com_df.sort_values(by = ['index', 'id'])

In [ ]:
calc_com_df.info()

In [ ]:
src_df.info()

In [ ]:
calc_com_df = pd.merge(src_df,  calc_com_df, how = 'left', on = ['index', 'id'])

In [ ]:
calc_com_df = pd.merge(complexity_java_df[['id', 'cdate']], calc_com_df, on = 'id')

In [ ]:
calc_com_df.head()

In [ ]:
# 일자별로 아예 소스가 수행되지 않는 건 
calc_com_df_back = calc_com_df.copy()
calc_ed_com_df = calc_com_df[~calc_com_df['file_nm'].isna()]

In [ ]:
calc_ed_com_df = calc_ed_com_df.sort_values(by = ['cdate', 'id', 'Cognitive Complexity'], ascending = [True, True, False]).groupby('id').head(1)

In [ ]:
calc_ed_com_df.head()

In [ ]:
calc_ed_com_df.info()

In [ ]:
calc_ed_com_df = calc_ed_com_df.groupby('cdate').mean(numeric_only=True)[['Cognitive Complexity']].reset_index()

In [ ]:
calc_ed_com_df.head()

In [ ]:
calc_ed_com_box_df = calc_ed_com_df.copy()

In [ ]:
calc_ed_com_box_df['yyyymm'] = calc_ed_com_box_df['cdate'].str[:7]

In [ ]:
ym_list = calc_ed_com_box_df['yyyymm'].unique()

In [ ]:
[calc_ed_com_box_df[calc_ed_com_box_df['yyyymm']==x]  for x in ym_list]

In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(1, 2, figsize = (20, 5), sharey=sharey, sharex=sharex)
# plt.xticks(rotation=45)
# major_xticks = ['2013-11-30','2014-11-30','2015-11-30','2016-11-30','2017-11-30','2018-11-30','2019-11-30','2020-11-30'
#                 ,'2021-11-30', '2022-11-30', '2023-11-30']
# yticks = np.arange(200, 2000, 400)
axs[0].bar(calc_ed_com_df['cdate']
        ,  calc_ed_com_df['Cognitive Complexity']
        ,  color = 'skyblue'
        # ,  edgecolor = 'black'
      ) # hist type

axs[0].set_title('Number of Questions over time', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('# of Q', fontsize = 15)
# axs[0].set_xticks(major_xticks)
# axs[0].set_yticks(yticks)
# axs[0].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)
# axs[0].yticks(yt)

axs[0].bar(calc_ed_com_df['cdate']
        ,  calc_ed_com_df['Cognitive Complexity']
        ,  color = 'skyblue'
        # ,  edgecolor = 'black'
      ) # hist type

axs[1].set_title('Number of Answers over time', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('# of A', fontsize = 15)
# axs[1].set_xticks(major_xticks)
# axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)

plt.show();


In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(1, 2, figsize = (20, 5), sharey=False, sharex=False)
# plt.xticks(rotation=45)
major_xticks = ['2022-01-30','2022-02-28','2022-03-30','2022-04-30','2022-05-30','2022-06-30','2022-07-30','2022-08-30','2022-09-30','2022-10-30','2022-11-30','2022-12-30',
                '2023-01-30','2023-02-28','2023-03-30','2023-04-30','2023-05-30','2023-06-30','2023-07-30','2023-08-30','2023-09-30','2023-10-30','2023-11-30','2023-12-30']
axs1_xticks = ym_list
axs1_xticks = np.insert(axs1_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 
# yticks = np.arange(200, 2000, 400)
axs[0].bar(calc_ed_com_df['cdate']
        ,  calc_ed_com_df['Cognitive Complexity']
        ,  color = 'skyblue'
        # ,  edgecolor = 'black'
      ) # hist type

axs[0].set_title('Avg Cognitive Complexity per Day', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('Avg Cognitive Complexity', fontsize = 15)
axs[0].set_xticks(major_xticks)
# axs[0].set_yticks(yticks)
# axs[0].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)
# axs[0].yticks(yt)

axs[1].boxplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list]
        # ,  edgecolor = 'black'
      ) # hist type

axs[1].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[1].set_xticks(range(len(axs1_xticks)), axs1_xticks)
# axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)

plt.show();


In [ ]:
ym_list[:11]
# ym_list[12:]

In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(2, 1, figsize = (10, 10), sharey=True, sharex=False)
# plt.xticks(rotation=45)

axs0_xticks = ym_list[:11]
axs0_xticks = np.insert(axs0_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 

axs1_xticks = ym_list[12:]
axs1_xticks = np.insert(axs1_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 
yticks = np.arange(0.5, 5, 0.5)

axs[0].boxplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[:11]]
        # ,  edgecolor = 'black'
      ) # hist type

axs[0].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[0].set_xticks(range(len(axs0_xticks)), axs0_xticks)
axs[0].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)


axs[1].boxplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[12:]]
        # ,  edgecolor = 'black'
      ) # hist type

axs[1].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[1].set_xticks(range(len(axs1_xticks)), axs1_xticks)
axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)
plt.tight_layout()    
plt.show();


In [ ]:
len(np.arange(2, 13, 1))

In [ ]:
sharey = True ## 또는 sharey=False
sharex = True ## 또는 sharex=False

fig, axs = plt.subplots(2, 1, figsize = (10, 10), sharey=True, sharex=False)
# plt.xticks(rotation=45)

axs0_xticks = ym_list[:11]
axs0_xticks = np.insert(axs0_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 

axs1_xticks = ym_list[12:]
axs1_xticks = np.insert(axs1_xticks, 0, '0')  # 인덱스 2 위치에 100을 추가 
yticks = np.arange(0.5, 5, 0.5)


axs[0].violinplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[:11]], positions=np.arange(1, 12, 1))
axs[0].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[0].set_xlabel('Date', fontsize = 15)
axs[0].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[0].set_xticks(range(len(axs0_xticks)), axs0_xticks)
axs[0].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[0].tick_params(axis='x', rotation=45)



axs[1].violinplot([calc_ed_com_box_df.loc[calc_ed_com_box_df['yyyymm']==x, 'Cognitive Complexity']  for x in ym_list[12:]], positions=np.arange(1, 12, 1))
axs[1].set_title('Cognitive Complexity per Month', fontsize = 15)
axs[1].set_xlabel('Date', fontsize = 15)
axs[1].set_ylabel('Cognitive Complexity', fontsize = 15)
axs[1].set_xticks(range(len(axs1_xticks)), axs1_xticks)
axs[1].set_yticks(yticks)
# axs[1].axvline([major_xticks[-2]], # add vertical line
#            color = 'blue',
#            linestyle = '--',
#            linewidth = 1.5)
axs[1].tick_params(axis='x', rotation=45)
plt.tight_layout()    
plt.show();
